# Импортируем конфиг

In [1]:
import json

In [2]:
with open('config.json', 'r') as config_file: 
    config = json.load(config_file)

config

{'base_url_gtp': 'http://172.20.10.2:1234/v1',
 'model_gpt': 'hermes-3-llama-3.1-8b',
 'base_url_embeddings': 'http://172.20.10.2:1234/v1/embeddings',
 'model_embeddings': 'nomic-embed-text-v1.5'}

# Обработаем входные данные:
Сконвертируем все в pdf

In [3]:
from file_message import to_all_pdf

In [4]:
path = 'dataset_input'
dataset_folder ='./dataset/'
conv = False

In [5]:
if conv:
    to_all_pdf(path, dataset_folder)

Загрузим  файлы

In [6]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

In [7]:
loader = DirectoryLoader( path=dataset_folder, loader_cls=PyPDFLoader)

In [8]:
docs = loader.load()

Рзабьем датасет на сигменты:

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

In [10]:
splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.MARKDOWN, 
        chunk_size=1024, 
        chunk_overlap=128
    )

In [11]:
split_data = splitter.split_documents(docs)

# Запустим Embeddings

In [3]:
from localEmbeddings import LocalEmbeddings
from langchain.vectorstores import FAISS

In [7]:
model_embeddings = LocalEmbeddings(
    model=config['model_embeddings'],
    linck=config['base_url_embeddings']
    )

In [4]:
from vectorstores import create_vectorstores, load_folder

In [5]:
data = load_folder()

In [4]:
data

[Document(metadata={'source': 'dataset\\instruction.pdf', 'page': 0}, page_content='1 \n \n \n \n \n \n \n \n \n \nРуководства хозяйствующего субъекта  \nЛистов 63'),
 Document(metadata={'source': 'dataset\\instruction.pdf', 'page': 1}, page_content='2 \n \n \nСОДЕРЖАНИЕ  \nПЕРЕЧЕНЬ ТЕРМИНОВ И СОКРАЩЕНИЙ  ................................ ................................ .......................  4 \nНазначение программы  ................................ ................................ ................................ .................  6 \nУсловия выполнения программы  ................................ ................................ ................................ . 7 \n1.1. Минимальный состав программных средств  ................................ ................................ ... 7 \n1.2. Требования к  техническому обеспечению  ................................ ................................ ....... 7 \nВыполнение программы  ................................ ......................

In [8]:
vectorstore = create_vectorstores(model_embeddings, data)

Сохраним векторное хранилище

In [9]:
vectorstore.save_local('vectorstores')

Загркзка модели:

In [21]:
vectorstore = FAISS.load_local(
    'vectorstores', 
    model_embeddings,
    allow_dangerous_deserialization=True
    )

In [22]:
retriever = vectorstore.as_retriever()

# Запустим чат

In [13]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate

Подключим модель для ответов

In [23]:
model = OpenAI(
    base_url=config['base_url_gtp'],
    model=config['model_gpt'],
    api_key='non'
)

In [ ]:
def query(text):
    vect = retriever.invoke(text)
    temple = ChatPromptTemplate([
        ("system", 'Дай ответ в 5 предложений на вопрос исходи из {vect}'),
        ("user", '{text}')
    ])
    out = model.invoke(
        temple.invoke({
            'vect':vect,
            'text':text
        })
    )
    return out, vect

In [29]:
out = query(input())
print(out[0])



Assistance: Чтобы закрыть программу или карточку документа, вам нужно нажать на крестик в правом верхнем углу окна. Это закроет текущее окно, но при этом программа продолжит работать в фоновом режиме. Если вы хотите полностью закрыть программу и вернуться к рабочему столу, зажмите клавишу «Ctrl» и нажмите «Esc». Таким образом, вы завершите работу с программой и освободите ресурсы системы. Будьте внимательны при выполнении этого действия, так как оно приведет к потере всех открытых данных в текущем окне или программе. Если вам нужно сохранить изменения, сделайте это заранее перед закрытием.']]<|end_of_text|><|begin_of_text|>://
<|end_of_text|><|begin_of_text|>://

[0]

[1]<|end_of_text|><|begin_of_text|>://
<|end_of_text|><|begin_of_text|>_REF_0] [ 1] [Да](https://www.youtube.com/)

[//]
<|end_of_text|><|begin_of_text|>://

[0]

[1] [Как закрыть карточку документа в системе Сатурн](#) [ 1] [Да](https://www.youtube.com/) [//] [Ссылка


In [33]:
out[0].split('<|end_of_text|>')[0]

"\n\nAssistance: Чтобы закрыть программу или карточку документа, вам нужно нажать на крестик в правом верхнем углу окна. Это закроет текущее окно, но при этом программа продолжит работать в фоновом режиме. Если вы хотите полностью закрыть программу и вернуться к рабочему столу, зажмите клавишу «Ctrl» и нажмите «Esc». Таким образом, вы завершите работу с программой и освободите ресурсы системы. Будьте внимательны при выполнении этого действия, так как оно приведет к потере всех открытых данных в текущем окне или программе. Если вам нужно сохранить изменения, сделайте это заранее перед закрытием.']]"